# Test the endpoint

In [15]:
import json
import pandas as pd
import numpy as np

In [12]:
import boto3
import sagemaker

client = boto3.client('sagemaker-runtime')

ModuleNotFoundError: No module named 'boto3'

In [16]:

df = pd.read_csv("data/mtsamples.csv")

## Run Inference on deployed endpoint

In [17]:
idx = np.random.randint(len(df))
input_txt = df.loc[idx].transcription
category = df.loc[idx].medical_specialty
print(category)
print(input_txt)

 Cardiovascular / Pulmonary
SUBJECTIVE: , Review of the medical record shows that the patient is a 97-year-old female patient who has been admitted and has been treated for community acquired pneumonia along with COPD exacerbation.  The patient does have a longstanding history of COPD.  However, she does not use oxygen at her independent assisted living home.  Yesterday, she had made improvement since being here at the hospital.  She needed oxygen.  She was tested for home O2 and qualified for it yesterday also.  Her lungs were very tight.  She did have wheezes bilaterally and rhonchi on the right side mostly.  She appeared to be a bit weak and although she was requesting to be discharged home, she did not appear to be fit for it.,Overnight, the patient needed to use the rest room.  She stated that she needed to urinate.  She awoke, decided not to call for assistance.  She stated that she did have her nurse call light button next to her and she was unable to gain access to her walker. 

In [ ]:
endpoint_name = "28-2023-04-04-06-38-09-524"
content_type = "text/csv" # "application/json" #
accept = "application/json" #"text/csv"
# payload = "test"#json.dumps({"instances":["test"]})

response = client.invoke_endpoint(
    EndpointName=endpoint_name,
    # CustomAttributes=custom_attributes,
    ContentType=content_type,
    Accept=accept,
    Body=input_txt,
)

json.loads(response["Body"].read())


## Run batch inference (Transformer)

In [82]:
session.boto_region_name

'eu-west-3'

In [ ]:
session = sagemaker.Session()
default_bucket = session.default_bucket()

model_name = "22-2023-04-03-13-43-05-767"
input_data_path = f"s3://{default_bucket}/data/test.csv"
output_data_path = f"s3://{default_bucket}/data/out"
CONTENT_TYPE_CSV = 'text/csv'

iam = boto3.client('iam')
role_arn = iam.get_role(RoleName=f'101436505502-sagemaker-exec')['Role']['Arn']
model_package_arn = "arn:aws:sagemaker:eu-west-3:101436505502:" \
                        f"model-package/training-pipelineModelGroup/22"


# c = boto3.client('sagemaker')
# c.list_model_packages(ModelPackageGroupName="training-pipelineModelGroup")


transform_job = sagemaker.transformer.Transformer(
    model_name = model_name,
    instance_count = 1,
    instance_type = 'ml.g4dn.xlarge', # ml.g4dn.xlarge
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    base_transform_job_name='inference-pipelines-batch',
    sagemaker_session=session,
    accept = CONTENT_TYPE_CSV)

transform_job.transform(data = input_data_path, 
                        content_type = CONTENT_TYPE_CSV, 
                        split_type = 'Line')

In [75]:
model = sagemaker.ModelPackage(role=role_arn, model_package_arn=model_package_arn,
                         sagemaker_session=session)

transformer =model.transformer(
    instance_count = 1,
    instance_type = 'ml.g4dn.xlarge', # ml.g4dn.xlarge
    strategy = 'SingleRecord',
    assemble_with = 'Line',
    output_path = output_data_path,
    accept = CONTENT_TYPE_CSV,
    # region="eu-west-3"
)

INFO:sagemaker:Creating model with name: 22-2023-04-03-13-51-02-471


In [ ]:
transformer.transform(data = input_data_path, 
                        content_type = CONTENT_TYPE_CSV, 
                        split_type = 'Line')

# Test local

In [9]:
class MyTokenizer:
    def __init__(self, model_name="distilbert-base-uncased") -> None:
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

    def tokenize(self, txt_input):
        return self.tokenizer.encode(txt_input, padding="max_length", truncation=True)

In [20]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained(
        "distilbert-base-uncased",
        num_labels=40,
    )

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [66]:
input["attention_mask"].shape

torch.Size([1, 512])

In [71]:
tok = MyTokenizer()
# input = tok.tokenizer(input_txt, padding="max_length", return_tensors='pt', truncation=True)
input = tok.tokenizer(input_txt, return_tensors='pt')

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
input.to(device)
outputs = model(**input)
y_pred = torch.argmax(outputs.logits.cpu(), dim=1)
y_pred
# [config.MEDICAL_CATEGORIES[i.item()] for i in y_pred]

Token indices sequence length is longer than the specified maximum sequence length for this model (886 > 512). Running this sequence through the model will result in indexing errors


RuntimeError: The size of tensor a (886) must match the size of tensor b (512) at non-singleton dimension 1